# Data Staging
Extract, Transform, Load (ETL)

## Extract datasets into dataframes

Extract job posting information

In [ ]:
import pandas as pd
df = pd.read_csv('job_descriptions.csv')
df

Extract City Population information

In [ ]:
df_population = pd.read_csv('CityPopulation.csv')
df_population

Extract company headquarters country location and company size (number of employees)

In [ ]:
df_company = pd.read_csv('CompanyInformation.csv')
df_company

## Transform

In [ ]:
pd.options.mode.copy_on_write = True #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
print(df.dtypes)

Keep only the countries we want for our analysis

In [ ]:
# Define the values to keep in the "Country" column
desired_countries = ['USA', 'UK', 'Canada', 'France', 'Japan', 'Belgium', 'Australia', 'Spain', 'India', 'Germany', 'Singapore', 'Thailand', 'China', 'Portugal', 'Vietnam', 'Mauritius']

# Filter the DataFrame to keep only rows with desired countries
df = df[df['Country'].isin(desired_countries)]

Drop unrequired columns

In [ ]:
df = df.drop(columns = ['latitude', 'longitude', 'Contact Person', 'Contact', 'Job Description', 'Company Size'])
df

Renaming columns

In [ ]:
# Columns to rename in df
columns_to_rename = {
    'skills':'Skills',
    'location': 'City',
    'Preference': 'Gender Preference',
    'Role': 'Specialization',
}

df = df.rename(columns_to_rename, axis='columns')

# columns to rename in df_population
df_population.rename(columns={'City Population': 'Job City Population'}, inplace=True)

# column to rename in df_company
df_company.rename(columns={'Country': 'Company HQ Country'}, inplace=True)

Processing date values

In [ ]:
# convert date object to datetime
df['Job Posting Date'] = pd.to_datetime(df['Job Posting Date'])
df.dtypes

In [ ]:
# Extract the month and year from job posting date and add them to 2 separate columns
df['Day'] = df['Job Posting Date'].dt.day # extract day
df['Month'] = df['Job Posting Date'].dt.month # extract month
df['Year'] = df['Job Posting Date'].dt.year # extract year

Processing Salary Range and Years of Experience

In [ ]:
# Split salary range in to minimum salary and maximum salary
df[['Minimum Salary', 'Maximum Salary']] = df['Salary Range'].str.split('-', expand=True)

# Keep only integers, using a regex, replace every non digit character by a ""
df['Minimum Salary'] = df['Minimum Salary'].str.replace('[^\d]', '', regex=True).astype(int)
df['Maximum Salary'] = df['Maximum Salary'].str.replace('[^\d]', '', regex=True).astype(int)


In [ ]:
# Transform salary into $ instead of in thousands unit
df['Minimum Salary'] *= 1000
df['Maximum Salary'] *= 1000


In [ ]:
# Split Years of experience range in to minimum experience and maximum experience
df[['Minimum Experience (years)', 'Maximum Experience (years)']] = df['Experience'].str.split('to', expand=True)

# Keep only integers, using a regex, replace every non digit character by a ""
df['Minimum Experience (years)'] = df['Minimum Experience (years)'].str.replace('[^\d]', '', regex=True).astype(int)
df['Maximum Experience (years)'] = df['Maximum Experience (years)'].str.replace('[^\d]', '', regex=True).astype(int)


In [ ]:
# Drop unneeded columns
df = df.drop(columns=['Experience', 'Salary Range', 'Job Posting Date'])
df

Handling null values within company profile



In [ ]:
# Check if there are null values, no null values if we successfully added missing companies
df.isnull().sum()

In [ ]:
# Checking which company has missing information on company profile
null_company_profile_rows = df[df['Company Profile'].isnull() | (df['Company Profile'] == '')]

# Extract unique companies from these rows
unique_companies_with_null_profile = null_company_profile_rows['Company'].unique()

print(unique_companies_with_null_profile)

In [ ]:
# handling missing values for company profile
import json
value_mapping = {
    'Estée Lauder': {"Sector":"Consumer Goods","Industry":"Consumer Goods","City":"New York","State":"New York","Zip":"10001","Website":"www.elcompanies.com","Ticker":"EL","CEO":"Fabrizio Freda"},
    'Dunkin\'Brands Group, Inc.': {"Sector":"Restaurants","Industry":"Food Services","City":"Canton","State":"Massachusetts","Zip":"02021","Website":"www.dunkindonuts.com","Ticker":"DNKN","CEO":"Nigel Travis"},
    'Peter Kiewit Sons': {"Sector":"Construction/Infrastructure","Industry":"Construction/Infrastructure","City":"Omaha","State":"Nebraska","Zip":"68102","Website":"www.kiewit.com","Ticker":"N/A","CEO":"Rick Lanoha"},
}

# Filling in missing information for company profile
for index, row in df.iterrows():
    if pd.isnull(row['Company Profile']):
        company = row['Company']
        if company in value_mapping:
          company_profile_str = json.dumps(value_mapping[company])
          df.at[index, 'Company Profile'] = company_profile_str

In [ ]:
# Checking if missing data was filled in properly

df.isnull().sum()

Cleaning data for Company Profile

In [ ]:
# Cleaning data that is formatted incorrectly in Company Profile
rows_to_replace = df[df['Company'] == 'Quanta Services']
df.loc[rows_to_replace.index, 'Company Profile'] = df.loc[rows_to_replace.index, 'Company Profile'].str.replace('"Duke" Austin', 'Austin')

Processing the Company Profile column

Format for Company Profile is a dictionnary, and we want to have a column for each key of the dictionnary and fill the column with the value associated to the key

In [ ]:
import ast

def extract_and_rename(dictionary_str, key, new_key):
    try:
        dictionary = ast.literal_eval(dictionary_str)
        value = dictionary.get(key, None)
        return value if value is not None else None
    except (SyntaxError, ValueError):
        return None

# Define keys to extract and their column names
key_to_new_key_mapping = {
    'Sector': 'Company Sector',
    'Industry': 'Company Industry',
    'City': 'Company HQ City',
    'Ticker': 'Company Ticker'
}

# Create new columns and fill with values
for key, new_key in key_to_new_key_mapping.items():
    df[new_key] = df['Company Profile'].apply(lambda x: extract_and_rename(x, key, new_key))

# Drop the original Company Profile column
df.drop(columns=['Company Profile'], inplace=True)
df

Cleaning the Company HQ City column

We noticed that some values for the City key included the city inside of it, so for example we have "City": "London, UK", but we only want the London part in that column

In [ ]:
# We split at the comma
split_cities = df['Company HQ City'].str.split(',')

# Filtering elements with length 2
split_cities_with_length_2 = split_cities[split_cities.apply(lambda x: len(x) == 2)]

# Getting indices of rows with split cities of length 2
indices_with_length_2 = split_cities_with_length_2.index

# Replacing the existing values with only the first element of the split list
df.loc[indices_with_length_2, 'Company HQ City'] = split_cities_with_length_2.apply(lambda x: x[0])

In [ ]:
# Checking if there are still rows with the format where we have the country, if the output returns nothing, then we cleaned properly

# Splitting the content of the 'Company HQ City' column at the comma
split_cities = df['Company HQ City'].str.split(',')

# Filtering elements with length 2
split_cities_with_length_2 = split_cities[split_cities.apply(lambda x: len(x) == 2)]

# Displaying the split content with elements of length 2
print(split_cities_with_length_2)

Processing the Benefits values

Format of Benefits table is {'Retirement Plans', 'Parental Leave', etc} and we want to have a column for each benefit and if the benefit is in the Object, we have True, and if it's not there we have False. So with the Object given above, Retirement Plans and Parental Leave will have True in their column and the others will have False

In [ ]:
benefit_columns = ['Retirement Plans','Stock Options or Equity Grants','Parental Leave','Paid Time Off (PTO)',
                   'Flexible Work Arrangements','Health Insurance','Life and Disability Insurance',
                   'Employee Assistance Program','Health and Wellness Facilities','Employee Referral Program',
                   'Transportation Benefits','Bonuses and Incentive Programs']

# Initialize all columns with False
for column in benefit_columns:
    df[column] = False

# Function to set boolean values
def set_benefit_values(row):
    benefits = row['Benefits']
    for column in benefit_columns:
        if column in benefits:
            row[column] = True
    return row

# Apply function to each row
df = df.apply(set_benefit_values, axis=1)

# Drop the original Benefits column
df.drop(columns=['Benefits'], inplace=True)
df

In [ ]:
# Checking if we have null values
df.isnull().sum()

Processing "N/A" values in Company Ticker column and replacing it with empty string. Only a small percentage of our data has a null value for this column, since this is a very small percentage, we ignore those missing values.

In [ ]:
df['Company Ticker'] = df['Company Ticker'].replace('N/A', '')

Cleaning strings in Company HQ City column that have special characters and aren't being displayed. This is happening for German cities.

In [ ]:
replacements = {
    'G ttingen': 'Göttingen',
    'Bad Homburg vor der H he': 'Bad Homburg vor der Höhe',
    'Unterf hring': 'Unterföhring',
    'Unterschlei heim': 'Unterschleißheim',
    'D sseldorf': 'Düsseldorf'
}

# Perform replacements
df['Company HQ City'] = df['Company HQ City'].replace(replacements, regex=True)

Create a column to represent Quarter attribute.

In [ ]:
# Add a 'Quarter' column based on 'Month'
df['Quarter'] = df['Month'].apply(lambda x: (x - 1) // 3 + 1)

### Dataset Integrations

We integrate our 3 datasets together: job_descriptions.csv, CityPopulation.csv and CompanyInformation.csv

In [ ]:
df = pd.merge(df, df_population, how='left', on=["City", "Country"])
df = pd.merge(df, df_company, how='left', on=["Company"])

In [ ]:
# Remove comma from population
df['Job City Population'] = df['Job City Population'].str.replace(',', '')

# Convert the City population and Company size to integer format
df['Job City Population'] = df['Job City Population'].astype('int64')
df['Company Size'] = df['Company Size'].astype('int64')

df.head(5)

In [ ]:
print(df.dtypes)

Reorder Columns

In [ ]:
# Reorder columns based on the desired order
desired_order = ['Job Id','Minimum Experience (years)',
                 'Maximum Experience (years)','Qualifications',
                 'Minimum Salary', 'Maximum Salary', 'City', 'Country', 'Job City Population',
                 'Work Type', 'Day', 'Month', 'Year', 'Quarter', 'Gender Preference', 'Job Title', 'Specialization',
                 'Job Portal', 'Skills', 'Responsibilities', 'Company', 'Company Size', 'Company Sector',
                 'Company Industry', 'Company HQ City', 'Company HQ Country', 'Company Ticker',
                 'Retirement Plans', 'Stock Options or Equity Grants', 'Parental Leave','Paid Time Off (PTO)',
                 'Flexible Work Arrangements','Health Insurance', 'Life and Disability Insurance',
                 'Employee Assistance Program','Health and Wellness Facilities','Employee Referral Program',
                 'Transportation Benefits','Bonuses and Incentive Programs']

df = df[desired_order]
df.head(5)

## Load

### Surrogate Key Generation

In [ ]:
# Create a new column new_result with sequential indices for each row
df['Surrogate Keys'] = range(1,len(df)+1)

In [ ]:
# after generation, surrogate key column is at the end of the dataset
# this code brings it to the beginning
df = df.reindex(columns=['Surrogate Keys'] + list([c for c in df.columns if c!= 'Surrogate Keys']))

In [ ]:
df.head(5)

### Saving the fully staged data for loading into a csv

In [ ]:
 # convert it back to csv
df.to_csv('Staged_data.csv')